<h1 style="font-size:3rem;color:maroon;"> Predicting Air Pollution Level using Machine Learning</h1>

This notebook looks into using various Python-based machine learning and data science libraries in an attempt to build a machine learning model capable of predicting air pollution level in an area in Eindhoven in the upcoming week.

We"re going to take the following approach:
1. Problem definition
2. Data
3. Features
4. Data Exploration & Visualization
5. Data Preparation
6. Modelling

<h2><font color=slateblue> 1. Problem Definition </font></h2>

In a statement,
> Given historical pollution data, weather data and people going through an area, can we predict air pollution level (fine particle matter level pm2.5) in an area in Eindhoven in the upcoming week?

<h2><font color=slateblue> 2. Data </font></h2>

The data is provided by TNO and Zicht op Data.

<h2><font color=slateblue> 3. Features </font></h2>

This is where you"ll get different information about each of the features in our data.

We have three separate datasets for the period between 25-09-2021 and 30-12-2021:

**Air pollution**
* date: date in ymd_hms
* PC4: postcode
* pm2.5: particulate matter <2.5um in ug/m3
* pm10: particulate matter <10um in ug/m3
* no2: nitrogen dioxide in ug/m3
* no: nitrogen oxide in ug/m3
* so2: sulphur dioxide in ug/m3


**Meteo**
* date: date in ymd_hms
* PC4: postcode
* wd: wind direction in degrees 0-360
* ws: wind speed in m/s
* blh: boundary layer height in metres
* tcc: total cloud cover in oktas (0-9)
* ssrd: solar surface radiation downwards in W/m2 

(see https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels?tab=overview for more information)

**Zichtop**
* PC4: postcode
* date: date in ymd_hms
* pop_tot: total number of people in PC4 for each time step
* m00_30: number of people who have been there for up to 30 minutes
* m30_60: number of people who have been there for 30 and 60 minutes
* H1_2: number of people who have been there for 1 and 2 hours
* H2_4: number of people who have been there for 2 and 4 hours
* H4_8: number of people who have been there for 4 and 8 hours
* H8_16: number of people who have been there for 8 and 16 hours
* H16plus: number of people who have been there for over 16 hours

<h2><font color=slateblue> Preparing the tools </font></h2>

In [ ]:
# Regular EDA (exploratory data analysis) and plotting libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Set the style
plt.style.use("ggplot")
# plt.style.available

<h2><font color=slateblue> 4. Data Exploration & Visualization </font></h2>

<h3><font color=steelblue>Zicht op Data dateset </font></h3>

<h4><font color=mediumvioletred>Read CSV files </font></h4>

In [ ]:
# read zichtop dataset csv file
df_zichtop = pd.read_csv("data/zichtop.csv", 
                    parse_dates=["date"])

# read air_pollution dataset csv file
df_air_pollution = pd.read_csv("data/air_pollution.csv",
                    parse_dates=["date"])

<h4><font color=mediumvioletred>Get a sample </font></h4>

In [ ]:
# zichtop sample
df_zichtop.head(10)

In [ ]:
# air_pollution sample
df_air_pollution.sample(5)

<h4><font color=mediumvioletred>Get number of rows and columns </font></h4>

In [ ]:
df_zichtop.shape

<h4><font color=mediumvioletred>Get types of columns </font></h4>

In [ ]:
df_zichtop.dtypes

<h4><font color=mediumvioletred>Get some info about each column (type, number of null values..) </font></h4>

In [ ]:
df_zichtop.info()

<h4><font color=mediumvioletred>Get some info about numerical columns (count, mean, min...) </font></h4>

In [ ]:
df_zichtop.describe()

<h4><font color=mediumvioletred>Merge zichtop and air_pollution datasets </font></h4>

In [ ]:
df_zichtop_air_pollution = pd.merge(df_zichtop, df_air_pollution[["PC4","date", "pm2.5"]], on=["PC4", "date"])
df_zichtop_air_pollution.sample(5)

In [ ]:
df_zichtop_air_pollution.dtypes

<h4><font color=mediumvioletred>Reorder columns in zichtop air pollution dataset </font></h4>

In [ ]:
zichtop_air_pollution_features = [
    "PC4",
    "date",
    "pop_tot",
    "pm2.5",
    "m00_30",
    "m30_60",
    "H1_2",
    "H2_4",
    "H4_8",
    "H8_16",
    "H16plus"
]

df_zichtop_air_pollution = df_zichtop_air_pollution.reindex(zichtop_air_pollution_features, axis=1)

df_zichtop_air_pollution.sample(5)

<h4><font color=mediumvioletred>Rename the date column to date_time </font></h4>

In [ ]:
df_zichtop_air_pollution.rename(columns={"date": "date_time"}, inplace=True)

<h4><font color=mediumvioletred>Separate date_time column to date and time </font></h4>

In [ ]:
df_zichtop_air_pollution["date"] = df_zichtop_air_pollution["date_time"].dt.date.astype(str)
df_zichtop_air_pollution["time"] = df_zichtop_air_pollution["date_time"].dt.time.astype(str)

In [ ]:
df_zichtop_air_pollution.sample(5)

In [ ]:
df_zichtop_air_pollution[(df_zichtop_air_pollution["PC4"] == 5611) & (df_zichtop_air_pollution.date == "2021-09-25")]

In [ ]:
# df_zichtop[(df_zichtop["PC4"] == 5611) & (df_zichtop.date == "2021-01-04")]
df_zichtop_air_pollution[(df_zichtop_air_pollution["PC4"] == 5611)]

<h4><font color=mediumvioletred>Get number of people in zipcode 5611 on 2021-09-25 </font></h4>

In [ ]:
people_5611 = df_zichtop_air_pollution[(df_zichtop_air_pollution["PC4"] == 5611) & (df_zichtop_air_pollution["date"] == "2021-09-25")]

<h4><font color=mediumvioletred>Visualize number of people in zipcode 5611 on 2021-09-25 </font></h4>

In [ ]:
fig, ax = plt.subplots(figsize=(25, 6))
# Plot the data
scatter = ax.bar(list(people_5611["time"]),
                list(people_5611["pop_tot"]),
                color="salmon");

# Customize the plot
ax.set(title="Number of people in zipcode: 5611 on 2021-09-25 per hour \n",
      xlabel="Time",
      ylabel="Number of People");

<h4><font color=mediumvioletred> Get maximum number of people per zipcode </font></h4>

In [ ]:
max_people = df_zichtop_air_pollution.groupby(["PC4"])["pop_tot"].agg("max").reset_index()
max_people.head()

<h4><font color=mediumvioletred> Visualize maximum number of people per zipcode </font></h4>

In [ ]:
fig, ax = plt.subplots(figsize=(30, 7))
# Plot the data
scatter = ax.bar(max_people["PC4"].astype(str),
                max_people["pop_tot"],
                color="salmon");

# Customize the plot
ax.set(title="Max number of people per zipcode \n",
      xlabel="Zipcode",
      ylabel="Number of People");

Zipcodes 5611, 5612, 5616 have the highest number of people.

<h4><font color=mediumvioletred> Get maximum, average and minimum number of people per zipcode </font></h4>

In [ ]:
max_mean_min_people = df_zichtop_air_pollution.groupby(["PC4"])["pop_tot"].agg(["max", "min", "mean"]).reset_index()
max_mean_min_people.head()

<h4><font color=mediumvioletred> Visualize maximum, average and minimum number of people per zipcode </font></h4>

In [ ]:
fig, ax = plt.subplots(figsize=(45, 10))

x = np.arange(len(max_mean_min_people["PC4"]))  # the label locations
width = 0.28  # the width of the bars

# Plot the data
rects1 = ax.bar(x - width, max_mean_min_people["max"], width, label="Maximum", color="salmon")
rects2 = ax.bar(x, max_mean_min_people["mean"], width, label="Average", color="lightblue")
rects3 = ax.bar(x + width, max_mean_min_people["min"], width, label="Minimum", color="mediumaquamarine")

# Customize the plot
ax.set_ylabel("Number of People")
ax.set_xlabel("Zipcode")
ax.set_title("Maximum, Average and Minimum number of people per zipcode \n")
ax.set_xticks(x, max_mean_min_people["PC4"])
ax.legend()

ax.bar_label(rects1, padding=4)
ax.bar_label(rects2, padding=4)
ax.bar_label(rects3, padding=4)

fig.tight_layout()

plt.show()

Zipcodes 5611, 5612, 5616 have the highest number of people and zipcodes 5633, 5656, 5617 have the lowest number of people.

<h4><font color=mediumvioletred> Get number of people vs. air quality </font></h4>

In [ ]:
# get data for zipcode 5611 on 2021-09-25
people_air_quality_area = df_zichtop_air_pollution[(df_zichtop_air_pollution["PC4"] == 5611) & (df_zichtop_air_pollution["date"] == "2021-09-25")]
people_air_quality_area.head(5)

<h4><font color=mediumvioletred> Visualize number of people vs. air quality </font></h4>

In [ ]:
# Number of people vs. fine particulate matter in a day in a specific area
fig, ax = plt.subplots(figsize=(10, 6))

# Plot the data
scatter = ax.scatter(x=people_air_quality_area["pop_tot"],
                    y=round(people_air_quality_area["pm2.5"]),
                    cmap="winter"); # color map: this changes the color scheme

# Customize the plot
ax.set(title="Number of people vs. fine particulate matter \n throughout the day for zipcode 5611 on 2021-09-25 \n",
      xlabel="Number of people",
      ylabel="Fine particulate matter");

# Add a horizontal line
ax.axhline(people_air_quality_area["pm2.5"].mean(),
          linestyle="--");

In [ ]:
# Subplot of number of people, pollution, time
fig, (ax0, ax1) = plt.subplots(nrows=2,
                                ncols=1,
                                figsize=(30, 10),
                                sharex=True)

# Add data to ax0
ax0.bar(people_air_quality_area["time"], people_air_quality_area["pop_tot"], color="salmon");
# Add data to ax1
ax1.bar(people_air_quality_area["time"], people_air_quality_area["pm2.5"], color="mediumaquamarine");

# Customize ax0
ax0.set(title="Number of people per hour \n",
      ylabel="Number of people");

# Customize ax1
ax1.set(title="Fine particulate matter per hour \n",
        xlabel="Time",
        ylabel="Fine particulate matter");

# Add a title to the figure
fig.suptitle("Air Quality Analysis for zipcode 5611 on 2021-09-25", fontsize=16, fontweight="bold");

<h4><font color=mediumvioletred> Get maximum, average and minimum fine particulate matter per zipcode </font></h4>

In [ ]:
# Get daily average of pm2.5 per zipcode
average_pm_per_day = df_zichtop_air_pollution.groupby(["PC4", "date"])["pm2.5"].agg({"mean"}).reset_index()

# Get maximum, average and minimum of pm2.5 per zipcode
max_mean_min_pm = average_pm_per_day.groupby(["PC4"])["mean"].agg(["max", "min", "mean"]).reset_index()
max_mean_min_pm.head()

<h4><font color=mediumvioletred> Visualize maximum, average and minimum fine particulate matter per zipcode </font></h4>

In [ ]:
fig, ax = plt.subplots(figsize=(45, 10))

x = np.arange(len(max_mean_min_pm["PC4"]))  # the label locations
width = 0.28  # the width of the bars

# Plot the data
rects1 = ax.bar(x - width, round(max_mean_min_pm["max"]), width, label="Maximum", color="salmon")
rects2 = ax.bar(x, round(max_mean_min_pm["mean"]), width, label="Average", color="lightblue")
rects3 = ax.bar(x + width, round(max_mean_min_pm["min"]), width, label="Minimum", color="mediumaquamarine")

# Customize the plot
ax.set_ylabel("Fine particulate matter")
ax.set_xlabel("Zipcode")
ax.set_title("Maximum, average and minimum fine particulate matter per zipcode \n")
ax.set_xticks(x, max_mean_min_pm["PC4"])
ax.legend()

ax.bar_label(rects1, padding=4)
ax.bar_label(rects2, padding=4)
ax.bar_label(rects3, padding=4)

fig.tight_layout()

plt.show()

According to the following diagram the maximum pm2.5 for each zipcode are on a moderate/unhealthy level and the average and minimum pm2.5 are on a good level.

![title](images/pm2.5_chart.jpg)

<h2><font color=slateblue> 5. Data Preparation </font></h2>

<h2><font color=slateblue> 6. Modelling </font></h2>